In [165]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import fred_msa

api_key = 'a37b50cd27afbc3ce23a81ddc5541dec'
endpoint = 'https://api.stlouisfed.org/fred/series/categories'

state_ids = fred_msa.get_state_ids(api_key)

msa_ids = pd.DataFrame()
for state, state_id in state_ids.items():
    state_dict = fred_msa.get_msa_cats(api_key, state_id)
    states = fred_msa.extract_state(pd.Series(state_dict.keys()))
    state_df = pd.DataFrame({'msa' : list(state_dict.keys()), 'state' : states, 'ID' : list(state_dict.values())})
    msa_ids = msa_ids.append(state_df)
    time.sleep(3)

msa_ids['city'] = fred_msa.split_city(msa_ids.msa)
msa_ids = msa_ids[msa_ids.msa.str.contains('CMSA') == False]
msa_ids = msa_ids.drop_duplicates(subset = ['city', 'state']).reset_index(drop = True)

# #msas_to_use = pd.read_csv('..\\fred-data\msas-to-use.csv', usecols=['city', 'state'])

# #msas_to_use = pd.merge(msas_to_use, msa_ids,
#          how='left', on=['city', 'state'], validate='one_to_one')

# #series = pd.DataFrame()

# for idx in range(len(msas_ids)):
#     msa_series = fred_msa.get_msa_series_list(msas_ids.ID[idx], api_key)
#     msa_series['msa'] = msas_ids.msa[idx]
#     msa_series['city'] = msas_ids.city[idx]
#     msa_series['state'] = msas_ids.state[idx]
    
#     series = series.append(msa_series).reset_index(drop=True)
#     time.sleep(3)
    

# series.to_csv('..\\fred-data\msa_series.csv', index=False)

<ipython-input-165-a6bb967f0ec0>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  states = fred_msa.extract_state(pd.Series(state_dict.keys()))


In [168]:
series = pd.DataFrame()

for idx in range(len(msa_ids)):
    msa_series = fred_msa.get_msa_series_list(msa_ids.ID[idx], api_key)
    msa_series['msa'] = msa_ids.msa[idx]
    msa_series['city'] = msa_ids.city[idx]
    msa_series['state'] = msa_ids.state[idx]
    
    series = series.append(msa_series).reset_index(drop=True)
    time.sleep(3)

In [170]:
keyword_list = ['Per Capita Personal Income', 'Resident Population', 'Unemployment Rate',
                'New Private Housing Units Authorized by Building Permits',
                'Regional Price Parities']

ny = series.query("city == 'New York'")
ny_housing = ny.title[ny.title.str.contains('Housing Inventory')]

housing_series = []
for s in ny_housing:
    if len(s.split(' in ')) > 2:
        housing_series.append(s.split(' in ')[0] + ' in ' + s.split(' in ')[1])
    else:
        housing_series.append(s.split(' in ')[0])

ny_employees = ny.title[ny.title.str.contains('All Employees')]

employees_series = []
for s in ny_employees:
    if len(s.split(' in ')) > 2:
        employees_series.append(s.split(' in ')[0] + ' in ' + s.split(' in ')[1])
    else:
        employees_series.append(s.split(' in ')[0])

keyword_list = keyword_list + housing_series + employees_series

In [185]:
clean_titles = []

for s in series.title:
    if len(s.split(' in ')) > 2:
        clean_titles.append(s.split(' in ')[0] + ' in ' + s.split(' in ')[1])
    else:
        clean_titles.append(s.split(' in ')[0])
        
series['clean_title'] = clean_titles

In [187]:
series.to_csv('..\\fred-data\\all_msa_series.csv', index=False)

In [188]:
relevant_series = series[series.clean_title.isin(keyword_list)]

In [219]:
all_relevant_series = relevant_series.drop_duplicates(['msa', 'clean_title'])
num_series = pd.DataFrame(all_relevant_series.groupby(['city', 'state'])['clean_title'].count())

In [230]:
num_by_cities = pd.DataFrame(all_relevant_series.groupby(['clean_title'])['msa'].count()).reset_index()

In [253]:
len(common_series_list)

58

In [248]:
pd.set_option('display.max_rows', 100)

num_by_cities.sort_values('msa', ascending=False).head(77)
common_series_list = list(num_by_cities[num_by_cities['msa'] > 300]['clean_title'])

In [341]:
common_series = series[series.clean_title.isin(common_series_list)].reset_index(drop=True)

all_common_series = common_series.drop_duplicates(['msa', 'clean_title']).reset_index(drop=True)
num_common_series = pd.DataFrame(all_common_series.groupby(['msa'])['clean_title'].count())

In [345]:
all_common_cities = pd.DataFrame(num_common_series[num_common_series.clean_title == 58]).reset_index()

In [347]:
all_common_cities = list(all_common_cities['msa'])

In [285]:
full_msas = num_common_series[num_common_series.clean_title == 58]

In [287]:
common_series['freq-adj'] = [common_series.loc[idx]['frequency'] + ' - '+ common_series.loc[idx]['seasonal_adjustment'] for idx in range(len(common_series))]

In [303]:
freq_adj = pd.DataFrame(common_series.groupby(['clean_title'])['freq-adj'].unique()).reset_index()
freq_adj['num'] = [len(freq_adj.loc[idx, 'freq-adj']) for idx in range(len(freq_adj))]

In [305]:
duplicates = freq_adj[freq_adj.num > 1]

In [325]:
duplicates['freq-adj'].astype('str').str.contains('Monthly - Seasonally Adjusted')
#freq_adj[freq_adj.num == 1]['freq-adj'].astype('str').str.contains('Monthly - Seasonally Adjusted')

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
57    True
Name: freq-adj, dtype: bool

In [337]:
common_series_list

['All Employees: Education and Health Services',
 'All Employees: Federal Government',
 'All Employees: Financial Activities',
 'All Employees: Goods Producing',
 'All Employees: Government',
 'All Employees: Government: Federal Government',
 'All Employees: Government: Local Government',
 'All Employees: Government: State Government',
 'All Employees: Information',
 'All Employees: Leisure and Hospitality',
 'All Employees: Local Government',
 'All Employees: Manufacturing',
 'All Employees: Mining, Logging, and Construction',
 'All Employees: Other Services',
 'All Employees: Private Service Providing',
 'All Employees: Professional and Business Services',
 'All Employees: Retail Trade',
 'All Employees: Service-Providing',
 'All Employees: State Government',
 'All Employees: Total Nonfarm',
 'All Employees: Total Private',
 'All Employees: Trade, Transportation, and Utilities',
 'Average Hourly Earnings of All Employees: Total Private',
 'Average Weekly Earnings of All Employees: To

In [258]:
monthly_series = common_series[(common_series.frequency == 'Monthly') & (common_series.seasonal_adjustment.str.contains('Not') == False)]

In [322]:
#monthly_series_counts = 
monthly_series.groupby(['msa']).clean_title.count().max()

26

In [271]:
monthly_series[monthly_series.title.str.contains('Housing')]

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes,msa,city,state,clean_title


In [350]:
monthly_series = monthly_series.drop_duplicates(['msa', 'clean_title'])
monthly_counts = pd.DataFrame(monthly_series.groupby(['msa'])['clean_title'].count()).reset_index()
cities_with_most_monthly = list(monthly_counts[monthly_counts.clean_title >= 23]['msa'])

In [355]:
cities_with_series = list(set(cities_with_most_monthly).intersection(set(all_common_cities)))

In [359]:
cities_with_housing = relevant_series.query("clean_title == 'Housing Inventory: Median Listing Price'")['msa'].unique()

In [362]:
cities_to_choose = list(set(cities_with_series).intersection(set(cities_with_housing)))

In [393]:
pop_keys = series[series.title.str.contains('Population')].reset_index(drop=True)

In [405]:
pops = pd.DataFrame()

In [406]:
pops

""


In [399]:
pop_keys = pop_keys[pop_keys.msa.isin(cities_to_choose)].reset_index(drop=True)

In [375]:
loc1 = fred_msa.get_series(pop_keys.loc[0, 'id'], api_key)


In [407]:
for idx in range(len(pop_keys)):
    pop_df = fred_msa.get_series(pop_keys.loc[idx, 'id'], api_key)
    pop_value = float(pop_df.tail(1).value)
    pops = pops.append(pd.DataFrame({'msa' : pop_keys.loc[idx, 'msa'],
                                    'pop': pop_value}, index=[idx]))
    time.sleep(3)

In [420]:
pop_sorted = pops.sort_values('pop', ascending=False).reset_index(drop=True)

In [421]:
top5 = list(pop_sorted.head(5)['msa'])

In [424]:
the_rest = pop_sorted[5:].reset_index(drop=True)

In [432]:
import random

In [446]:
selected_cities = top5.copy()
random.seed(500)
for i in range(5):
    start = int(np.floor((len(the_rest)/5)*i))
    end = int(np.floor((len(the_rest)/5)*(i+1)) - 1)
    idxs = random.sample(range(start, end), 5)
    _ = [selected_cities.append(the_rest.loc[idx].msa) for idx in idxs]
    

In [450]:
pd.DataFrame({'msa' : selected_cities}).to_csv('..\\fred-data\\new-msas-to-use.csv')

In [459]:
new_series = series[(series.clean_title.isin(common_series_list)) & (series.msa.isin(selected_cities))]

In [462]:
selected_cities

['New York-Newark-Jersey City, NY-NJ-PA',
 'Dallas-Fort Worth-Arlington, TX',
 'Houston-The Woodlands-Sugar Land, TX',
 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'Atlanta-Sandy Springs-Alpharetta, GA',
 'New Orleans-Metairie, LA',
 'Birmingham-Hoover, AL',
 'Tulsa, OK',
 'Kansas City, MO-KS',
 'Albuquerque, NM',
 'Wichita, KS',
 'Colorado Springs, CO',
 'Oxnard-Thousand Oaks-Ventura, CA',
 'Madison, WI',
 'Lancaster, PA',
 'York-Hanover, PA',
 'Reno, NV',
 'Reading, PA',
 'Beaumont-Port Arthur, TX',
 'Fort Wayne, IN',
 'Topeka, KS',
 'Lynchburg, VA',
 'Olympia-Lacey-Tumwater, WA',
 'Crestview-Fort Walton Beach-Destin, FL',
 'Charleston, WV',
 'Johnstown, PA',
 'Grand Forks, ND-MN',
 'Wichita Falls, TX',
 'Morristown, TN',
 'Panama City, FL']

In [463]:
pres_cities = ['New York-Newark-Jersey City, NY-NJ-PA', 'Atlanta-Sandy Springs-Alpharetta, GA', 'Topeka, KS', 'Tulsa, OK']

In [466]:
pres_keywords = ['All Employees: Education and Health Services',
 'All Employees: Federal Government',
 'All Employees: Financial Activities',
 'Housing Inventory: Median Days on Market',
 'Housing Inventory: Median Home Size in Square Feet',
 'Housing Inventory: Median Listing Price',
 'Housing Inventory: New Listing Count',
 'Per Capita Personal Income',
 'Resident Population',
 'Unemployment Rate']

In [470]:
pres_series = series[(series.clean_title.isin(pres_keywords)) & (series.msa.isin(pres_cities))]
len(pres_series)
pres_series.to_csv('..\\fred-data-pres\\pres-series.csv')

In [266]:
monthly_list = list(monthly_series.clean_title.unique())

In [267]:
monthly_list

['All Employees: Education and Health Services',
 'All Employees: Financial Activities',
 'All Employees: Government',
 'All Employees: Information',
 'All Employees: Leisure and Hospitality',
 'All Employees: Manufacturing',
 'All Employees: Total Nonfarm',
 'All Employees: Professional and Business Services',
 'All Employees: Other Services',
 'All Employees: Trade, Transportation, and Utilities',
 'Unemployment Rate',
 'All Employees: Total Private',
 'Average Weekly Hours of All Employees: Total Private',
 'Average Hourly Earnings of All Employees: Total Private',
 'Average Weekly Earnings of All Employees: Total Private',
 'All Employees: Goods Producing',
 'All Employees: Service-Providing',
 'All Employees: Private Service Providing',
 'All Employees: Mining, Logging, and Construction',
 'All Employees: Retail Trade',
 'All Employees: Government: Federal Government',
 'All Employees: Government: State Government',
 'All Employees: Government: Local Government',
 'All Employees: 

In [204]:
monthly_series.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes,msa,city,state,clean_title
3,ANNI501EDUH,2021-12-08,2021-12-08,All Employees: Education and Health Services i...,1990-01-01,2021-10-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2021-11-19 16:57:20-06,1.0,1.0,The Federal Reserve Bank of St. Louis seasonal...,"Anniston-Oxford, AL",Anniston,AL,All Employees: Education and Health Services
5,ANNI501FIRE,2021-12-08,2021-12-08,All Employees: Financial Activities in Annisto...,1990-01-01,2021-10-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2021-11-19 16:57:19-06,1.0,1.0,The Federal Reserve Bank of St. Louis seasonal...,"Anniston-Oxford, AL",Anniston,AL,All Employees: Financial Activities
7,ANNI501GOVT,2021-12-08,2021-12-08,All Employees: Government in Anniston-Oxford-J...,1990-01-01,2021-10-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2021-11-19 16:57:19-06,1.0,1.0,The Federal Reserve Bank of St. Louis seasonal...,"Anniston-Oxford, AL",Anniston,AL,All Employees: Government
9,ANNI501INFO,2021-12-08,2021-12-08,All Employees: Information in Anniston-Oxford-...,1990-01-01,2021-10-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2021-11-19 16:57:21-06,1.0,1.0,The Federal Reserve Bank of St. Louis seasonal...,"Anniston-Oxford, AL",Anniston,AL,All Employees: Information
11,ANNI501LEIH,2021-12-08,2021-12-08,All Employees: Leisure and Hospitality in Anni...,1990-01-01,2021-10-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2021-11-19 16:57:19-06,1.0,1.0,The Federal Reserve Bank of St. Louis seasonal...,"Anniston-Oxford, AL",Anniston,AL,All Employees: Leisure and Hospitality


In [212]:
len(monthly_series.clean_title.unique())

45

In [154]:
ny = series.query("city == 'New York'")
ny_housing = ny.title[ny.title.str.contains('Housing Inventory')]
housing_series = [s.split(' in')[0] for s in ny_housing]

In [158]:
housing_series

['Housing Inventory: Active Listing Count',
 'Housing Inventory: Active Listing Count Month-Over-Month',
 'Housing Inventory: Active Listing Count Year-Over-Year',
 'Housing Inventory: Average Listing Price',
 'Housing Inventory: Average Listing Price Month-Over-Month',
 'Housing Inventory: Average Listing Price Year-Over-Year',
 'Housing Inventory: Median Days on Market',
 'Housing Inventory: Median Days on Market Month-Over-Month',
 'Housing Inventory: Median Days on Market Year-Over-Year',
 'Housing Inventory: Median Listing Price',
 'Housing Inventory: Median Listing Price Month-Over-Month',
 'Housing Inventory: Median Listing Price per Square Feet',
 'Housing Inventory: Median Listing Price per Square Feet Month-Over-Month',
 'Housing Inventory: Median Listing Price per Square Feet Year-Over-Year',
 'Housing Inventory: Median Listing Price Year-Over-Year',
 'Housing Inventory: Median Home Size',
 'Housing Inventory: Median Home Size',
 'Housing Inventory: Median Home Size',
 'Hous

In [163]:
ny_employees = ny.title[ny.title.str.contains('All Employees')]
employees_series = list(np.unique([s.split(' in')[0] for s in ny_employees]))
employees_series

['All Employees: Accommodation and Food Services',
 'All Employees: Administrative and Support and Waste Management and Remediation Services',
 'All Employees: Education and Health Services',
 'All Employees: Education and Health Services: Health Care and Social Assistance',
 'All Employees: Federal Government',
 'All Employees: Financial Activities',
 'All Employees: Financial Activities: Finance and Insurance',
 'All Employees: Financial Activities: Insurance Carriers and Related Activities',
 'All Employees: Goods Producing',
 'All Employees: Government',
 'All Employees: Government: Federal Government',
 'All Employees: Government: Local Government',
 'All Employees: Government: State Government',
 'All Employees: Health Care and Social Assistance',
 'All Employees: Health Care: Ambulatory Health Care Services',
 'All Employees: Health Care: Hospitals',
 'All Employees: Information',
 'All Employees: Information: Telecommunications',
 'All Employees: Leisure and Hospitality',
 'All

In [164]:
state_ids

{'Alabama': 27282,
 'Alaska': 27283,
 'Arizona': 27284,
 'Arkansas': 149,
 'California': 27286,
 'Colorado': 27287,
 'Connecticut': 27288,
 'Delaware': 27289,
 'District of Columbia': 27290,
 'Florida': 27291,
 'Georgia': 27292,
 'Hawaii': 27293,
 'Idaho': 27294,
 'Illinois': 150,
 'Indiana': 151,
 'Iowa': 27297,
 'Kansas': 27298,
 'Kentucky': 152,
 'Louisiana': 27300,
 'Maine': 27301,
 'Maryland': 27302,
 'Massachusetts': 27303,
 'Michigan': 27304,
 'Minnesota': 27305,
 'Mississippi': 153,
 'Missouri': 154,
 'Montana': 27308,
 'Nebraska': 27309,
 'Nevada': 27310,
 'New Hampshire': 27311,
 'New Jersey': 27312,
 'New Mexico': 27313,
 'New York': 27314,
 'North Carolina': 27315,
 'North Dakota': 27316,
 'Ohio': 27317,
 'Oklahoma': 27318,
 'Oregon': 27319,
 'Pennsylvania': 27320,
 'Puerto Rico': 33504,
 'Rhode Island': 27322,
 'South Carolina': 27323,
 'South Dakota': 27324,
 'Tennessee': 193,
 'Texas': 27326,
 'Utah': 27328,
 'Vermont': 27329,
 'Virgin Islands': 33605,
 'Virginia': 27330